In [1]:
import os
import openai

import numpy as np
import pandas as pd
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

EMBEDDING_MODEL = 'text-embedding-ada-002'
GPT_MODEL = 'gpt-3.5-turbo-0301'
THRESHOLD_COSINE_SIMILARITY = 0.85

In [3]:
openai.api_key = OPENAI_API_KEY

In [4]:
def str_to_nparray(str_array):
    return np.fromstring(str_array[1:-1], sep=',')

df = pd.read_csv('./data/embedding_content.csv', converters={'embedding': str_to_nparray})
df.head(5)

,url,content,embedding
0,https://zozo.jp/_help/default.html?id=5ece39d3...,商品のサイズ商品のサイズは、商品ページ下部にある「サイズ」の項目をご覧ください。当サイトでは...,"[-0.012270285747945309, 0.009278957732021809, ..."
1,https://zozo.jp/_help/default.html?id=5ece39d3...,商品のカラー・デザイン商品の画像は、できる限り実際の商品に近づけた状態で掲載をしております。...,"[-0.012848308309912682, -0.0011484753340482712..."
2,https://zozo.jp/_help/default.html?id=5ece47a3...,商品の販売価格サイトの表示価格は、販売ショップがZOZOTOWNで販売するために設定した価格...,"[-0.015740083530545235, -0.006914299912750721,..."
3,https://zozo.jp/_help/default.html?id=5ece39d4...,ZOZOUSEDの取扱い商品ZOZOUSEDでは、買取サービスや買い替え割・いつでも買い替え...,"[-0.01777864620089531, -0.008320022374391556, ..."
4,https://zozo.jp/_help/default.html?id=5ece39d4...,ZOZOUSED商品の掲載画像撮影時の照明やカメラの角度、お客様の端末設定により、実際の色味...,"[-0.004656823817640543, -0.005725925322622061,..."


In [5]:
def get_embedding(text):
    return openai.Embedding.create(input=[text], model=EMBEDDING_MODEL)['data'][0]['embedding']

In [6]:
def get_cosine_similarity(array_a, array_b):
    return float(np.dot(array_a, array_b) / (np.linalg.norm(array_a) * np.linalg.norm(array_b)))

In [7]:
def get_similar_contents(input_text):
    embedding_input_text = get_embedding(input_text)

    df_cloned = df.copy(deep=True)
    df_cloned['cosine_similarity'] = df_cloned['embedding'].apply(lambda x: get_cosine_similarity(x, embedding_input_text))
    df_cloned_filtered = df_cloned[df_cloned['cosine_similarity'] >= THRESHOLD_COSINE_SIMILARITY]
    df_top = df_cloned_filtered.sort_values(['cosine_similarity'], ascending=False).iloc[: 3]

    results = []
    for i in range(len(df_top)):
        row = df_top.iloc[i]
        results += [{
            'url': row.get('url'),
            'content': row.get('content'),
            'cosine_similarity': row.get('cosine_similarity'),
        }]

    return results

In [8]:
def generate_answer(input_text):
    similar_contents = get_similar_contents(input_text)

    answer_conten = '\n\n'.join([
        '『{}』\nURL: {}'.format(content['content'], content['url'])
        for content in similar_contents
    ])

    system_prompt = f'あなたはマニュアルを参考にお問い合わせに回答するチャットボットです。ユーザーの質問に対してマニュアルを利用してMarkdown形式で回答してください。' \
        + f'また、参考にしたマニュアルのURLも箇条書きで教えてください。質問に沿った内容の回答が用意できない場合は、想像で回答せずにマニュアルに存在しない旨を回答してください。'

    prompt = f'【ユーザーの質問】' \
        + f'\n{input_text}' \
        + f'\n\n【マニュアル】' \
        + f'\n{answer_conten}'

    print(prompt)

    res = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[
            { 'role': 'system', 'content': system_prompt },
            { 'role': 'user', 'content': prompt },
        ]
    )

    print('\n\n==========\n\n')
    print(res.choices[0].message.content)

    # return res.choices[0].message.content

In [9]:
generate_answer('返品したいときは')

【ユーザーの質問】
返品したいときは

【マニュアル】
『返品時の返送料返品対応時に発生する返送料の扱いは下記のとおりです。※返品の際は事前にご申請いただく必要がございます。詳細はヘルプページ「返品の申請方法」をご確認ください。なお、返送に関する詳細は、返品申請後に届く返送案内メールにも記載がございますので、必ずあわせてご確認ください。※商品の返送が簡単にできるサービスもございます。詳細はヘルプページ「便利な返送サービス」をご確認ください。● お客様都合による返品：お客様負担（元払い） ∟サイズが合わなかった場合 ∟イメージが違った場合● 不備による返品：弊社負担（着払い） ∟不良があった場合 ∟注文と異なる商品が届いた場合 ∟商品ページに記載の情報に誤りがあった場合※キャンペーン等によって一時的に変更となる場合がございます。』
URL: https://zozo.jp/_help/default.html?id=624fc03a7b9e340022281e46

『返品ではなく交換したい / 返品申請を取り消したい【返品ではなく交換したい】返品理由が以下の場合、交換対応は承っておりません。「サイズが合わない」「イメージと違う」「注文を間違えた」交換をご希望の場合は、返品後に再注文をお願いいたします。詳細は「商品の変更・交換」をご確認ください。【返品申請を取消したい】取り消し手続きおよびお問い合わせは不要です。お手元の商品は、そのままお受け取りください。』
URL: https://zozo.jp/_help/default.html?id=649295783b2d33001b2d0fd3

『返品の申請方法「商品の返品」に記載のある返品交換条件を満たしている商品は、以下の手続きが可能です。1）「注文履歴注文履歴」内の「注文詳細・各種手続き」より「返品・交換の手続き」へ進み、手続きをお願いします。ログイン・会員登録せずに注文された場合は、注文後に届く注文確定メール内より「注文履歴」へ進むことが可能です。2）手続き完了後、会員登録メールアドレスへ返送のご案内メールを配信いたします。内容を確認のうえ、商品のご返送をお願いします。ログイン・会員登録せずに注文された方の場合、注文時にご入力のメールアドレスに配信いたします。	3）ご返送商品を確認後、7営業日以内に返金日などの

In [10]:
generate_answer('ポイントについて詳しく教えてください')

【ユーザーの質問】
ポイントについて詳しく教えてください

【マニュアル】
『ポイントの種類以下のポイントがZOZOTOWNで使用できます。【ZOZOCARDご利用還元ポイント】ZOZOCARDのご利用で加算されるポイントです。詳しくは「ZOZOCARDとは？ZOZOCARDとは？」をご確認ください。【買取査定ポイント】買取サービスにて、「ZOZOポイント買取」を希望された場合に加算されるポイントです。買取サービスに関しては、「ブランド古着買取サービスブランド古着買取サービス」をご確認ください。【ギフトポイント】LINEの友だちからプレゼントとして受け取れるポイントです。詳しくは「LINEギフトとは」をご確認ください。【コスメショップ限定ポイント】ZOZOCOSMEの専門ショップで販売中の商品にご利用いただけるポイントです。アパレルショップで販売されているコスメアイテムにはご利用いただけませんので、ご注意ください。【それ以外のポイント】特定のショップでご利用いただける「ショップ限定クーポンポイント」、キャンペーンにより付与される「キャンペーンポイント」などがございます。各ポイントによってご利用条件が異なりますので、ポイントの説明やキャンペーンの注意事項を必ずご確認ください。※一部、送料・各種手数料のお支払いにご使用いただけないポイントがございます。』
URL: https://zozo.jp/_help/default.html?id=5ece39d49ed84e001ea6e379




【マニュアルより回答】
ZOZOTOWNで使用できるポイントは以下のとおりです。

- ZOZOCARDご利用還元ポイント: ZOZOCARDでご利用いただくと加算されるポイントです。詳しくは「ZOZOCARDとは？」をご確認ください。
- 買取査定ポイント: 買取サービスにて、"ZOZOポイント買取"を希望された場合に加算されるポイントです。
- ギフトポイント: LINEの友だちからプレゼントとして受け取れるポイントです。
- コスメショップ限定ポイント: ZOZOCOSMEの専門ショップで販売中の商品にご利用いただけるポイントです。
- ショップ限定クーポンポイント, キャンペーンポイント, その他のポイント: 特定のショップでご利用いただける「ショップ限定クーポンポイン